In [2]:
#Linear Regression Draft

import pandas as pd
from sklearn.model_selection import train_test_split

from google.colab import files


uploaded = files.upload()

Saving restaurant-1-orders.csv to restaurant-1-orders (1).csv


In [4]:
import io

data = pd.read_csv(io.BytesIO(uploaded['restaurant-1-orders (1).csv']))
print(data)

       Order Number        Order Date              Item Name  Quantity  \
0             16118  03/08/2019 20:25          Plain Papadum         2   
1             16118  03/08/2019 20:25       King Prawn Balti         1   
2             16118  03/08/2019 20:25            Garlic Naan         1   
3             16118  03/08/2019 20:25          Mushroom Rice         1   
4             16118  03/08/2019 20:25    Paneer Tikka Masala         1   
...             ...               ...                    ...       ...   
74813          2518  18/05/2016 18:53    House Red wine 75cl         2   
74814          2503  14/05/2016 19:28    House Red wine 75cl         2   
74815          2467  10/05/2016 13:18    House Red wine 75cl         2   
74816          2249  07/04/2016 19:04  House white wine 75cl         1   
74817          1403  02/10/2015 17:29    House Red wine 75cl         1   

       Product Price  Total products  
0               0.80               6  
1              12.95             

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Convert 'Order Date' to datetime, considering day-first format
data['Order Date'] = pd.to_datetime(data['Order Date'], dayfirst=True, errors='coerce')

# Aggregate data by date and item name
daily_data = data.groupby(['Order Date', 'Item Name']).agg({
    'Quantity': 'sum'  # Sum quantities for each day and item
}).reset_index()

# Generate features from date
daily_data['day_of_week'] = daily_data['Order Date'].dt.dayofweek
daily_data['month'] = daily_data['Order Date'].dt.month

# Encode 'Item Name' using one-hot encoding
encoder = OneHotEncoder(sparse=False)
item_encoded = encoder.fit_transform(daily_data[['Item Name']])

# Convert encoded data to DataFrame
item_encoded_df = pd.DataFrame(item_encoded, columns=encoder.get_feature_names_out(['Item Name']))

# Concatenate the one-hot encoded columns back to the original DataFrame
daily_data_encoded = pd.concat([daily_data, item_encoded_df], axis=1)

# Select features and target
X = daily_data_encoded.drop(['Order Date', 'Item Name', 'Quantity'], axis=1)
y = daily_data_encoded['Quantity']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
#Model Training
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Initialize and train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


In [ ]:
#Save and Load the model

import joblib

# Save the model
joblib.dump(model, 'food_demand_forest_model.pkl')

# Load the model
loaded_model = joblib.load('food_demand_forest_model.pkl')

# Example: New data for prediction
new_data_raw = pd.DataFrame({
    'day_of_week': [1],  # Monday
    'month': [5],       # May
    'Item Name': ['Specific Dish Name']  # Example item
})

# Encode 'Item Name' in new data
new_item_encoded = encoder.transform(new_data_raw[['Item Name']])
new_item_encoded_df = pd.DataFrame(new_item_encoded, columns=encoder.get_feature_names_out(['Item Name']))

# Concatenate other features
new_data_encoded = pd.concat([new_data_raw.drop('Item Name', axis=1), new_item_encoded_df], axis=1)

# Use the loaded model to make predictions
new_prediction = model.predict(new_data_encoded)
print(f'Predicted demand: {new_prediction[0]}')

Predicted demand: 1.1808225418393492
